In [2]:
import shutil
import os
import io


shutil.unpack_archive(filename, extract_dir)


In [ ]:
filename = "dane_eeg.zip"
extract_dir = "dane_eeg"
seizure_occ_file = "dane_eeg/czasy_napadow.m"
healthy_dir = "dane_eeg/zdrowi"
diagnosed_dir = "dane_eeg/chorzy"

In [31]:

def get_mat_files(base_path): 
    mat_files_list = []
    filenames = {}

    for file in os.listdir(base_path):
        if file.endswith(".mat"):
            mat_files_list.append(base_path + "/" + file)
            
    for filename in mat_files_list:        
        filenames[filename.split('/')[-1].split('_')[0]] = filename
        
    return mat_files_list, filenames


healthy_paths, healthy_patients_dict = get_mat_files(healthy_dir)
diagnosed_paths, diagnosed_patients_dict = get_mat_files(diagnosed_dir)

In [8]:
import re

def get_all_occurrences(file_path):
    all_occurrences = {}
    with open (file_path , 'r') as file:
        for line in file :
            line_number = int(re.search(r'{([0-9]+)}', line).group(1))
            occ_list = list(map(int, re.search(r'\[(.+?)]', line).group(1).split()))
            all_occurrences[line_number] = occ_list
    return all_occurrences


occurences = get_all_occurrences(seizure_occ_file)

In [35]:
class Seizure:
    def __init__ (self, id, filename, samples, seizure_occurences=None, frequency=100, channels=16) -> None :
        super (). __init__ ()
        self.id = id
        self.filename = filename
        self.samples = samples
        self.seizure_occurences = seizure_occurences
        self.frequency = frequency
        self.channels = channels

In [36]:
def healthy_patients(healthy_people_dict):
    healthy_patients = []
    for id, path in healthy_people_dict.items():
        samples = io.loadmat(path)['x']
        filename = os.path.basename(path)
        healthy_people.append(Seizure(id, filename, samples))
 
    return healthy_patients

def unhealthy_patients(unhealthy_people_dict, seizure_occurences):
    unhealthy_patients = []
    for id, path in unhealthy_people_dict.items():
        seizure_occ = seizure_occurences[id]
        samples = io.loadmat(path)['x']
        filename = os.path.basename(path)
        unhealthy_people.append(Seizure(id, filename, samples, seizure_occurences=seizure_occ))
 
    return unhealthy_patients

39 dane_eeg/zdrowi/39_z_100Hz.mat
49 dane_eeg/zdrowi/49_z_100Hz.mat
17 dane_eeg/zdrowi/17_z_100Hz.mat
67 dane_eeg/zdrowi/67_z_100Hz.mat
55 dane_eeg/zdrowi/55_z_100Hz.mat
3 dane_eeg/zdrowi/3_z_100Hz.mat
25 dane_eeg/zdrowi/25_z_100Hz.mat
40 dane_eeg/zdrowi/40_z_100Hz.mat
30 dane_eeg/zdrowi/30_z_100Hz.mat
72 dane_eeg/zdrowi/72_z_100Hz.mat
35 dane_eeg/zdrowi/35_z_100Hz.mat
45 dane_eeg/zdrowi/45_z_100Hz.mat
59 dane_eeg/zdrowi/59_z_100Hz.mat
29 dane_eeg/zdrowi/29_z_100Hz.mat
62 dane_eeg/zdrowi/62_z_100Hz.mat
12 dane_eeg/zdrowi/12_z_100Hz.mat
20 dane_eeg/zdrowi/20_z_100Hz.mat
6 dane_eeg/zdrowi/6_z_100Hz.mat
50 dane_eeg/zdrowi/50_z_100Hz.mat
5 dane_eeg/zdrowi/5_z_100Hz.mat
23 dane_eeg/zdrowi/23_z_100Hz.mat
53 dane_eeg/zdrowi/53_z_100Hz.mat
61 dane_eeg/zdrowi/61_z_100Hz.mat
11 dane_eeg/zdrowi/11_z_100Hz.mat
36 dane_eeg/zdrowi/36_z_100Hz.mat
46 dane_eeg/zdrowi/46_z_100Hz.mat
18 dane_eeg/zdrowi/18_z_100Hz.mat
68 dane_eeg/zdrowi/68_z_100Hz.mat
9 dane_eeg/zdrowi/9_z_100Hz.mat
71 dane_eeg/zdrowi/71_